<a href="https://colab.research.google.com/github/wakepon21/Masa/blob/master/clrp_pytorch_roberta_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses the model created in pretrain any model notebook.

This is inference notebooks that is trained using below notebooks.

This notebook uses the model created in pretrain any model notebook.


In [3]:
import sys
from pathlib import Path

if 'google.colab' in sys.modules:  # colab環境
    INPUT = "/content/drive/My Drive/kaggle/commonlit/input/"
    import os
    import json

    !pip install kaggle
    !pip install transformers
    !git clone https://github.com/huggingface/transformers
    %cd transformers
    !pip install colorama
    from transformers import RobertaTokenizer, RobertaTokenizerFast

    from google.colab import drive
    drive.mount('/content/drive')
    f = open("/content/drive/My Drive/kaggle.json", 'r')
    json_data = json.load(f) 
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']
    OUTPUT="/content/drive/My Drive/kaggle/commonlit/output/finetuning"
    

elif 'kaggle_web_client' in sys.modules:  # kaggle環境
    INPUT = "../input/"
    OUTPUT="./"
    import transformers
    from transformers import *

     |████████████████████████████████| 2.3MB 5.3MB/s 
     |████████████████████████████████| 901kB 45.0MB/s 
     |████████████████████████████████| 3.3MB 48.7MB/s 
Cloning into 'transformers'...
remote: Enumerating objects: 75053, done.
remote: Counting objects: 100% (451/451), done.
remote: Compressing objects: 100% (335/335), done.
remote: Total 75053 (delta 180), reused 257 (delta 98), pack-reused 74602
Receiving objects: 100% (75053/75053), 58.62 MiB | 25.06 MiB/s, done.
Resolving deltas: 100% (53337/53337), done.
/content/transformers
Mounted at /content/drive


In [4]:
import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import (AutoModel, AutoTokenizer,AdamW,
                          AutoConfig,AutoModelForSequenceClassification,
                          get_linear_schedule_with_warmup,
                          get_cosine_schedule_with_warmup)

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [5]:
train_data = pd.read_csv(INPUT+'commonlitreadabilityprize/train.csv')
test_data = pd.read_csv(INPUT+'commonlitreadabilityprize/test.csv')
sample = pd.read_csv(INPUT+'commonlitreadabilityprize/sample_submission.csv')

num_bins = int(np.floor(1 + np.log2(len(train_data))))
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)

bins = train_data.bins.to_numpy()
target = train_data.target.to_numpy()

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [6]:
config = {
    'lr': 2e-5,
    'wd':0.01,
    'batch_size':16,
    'valid_step':10,
    'max_len':256,
    'epochs':3,
    'nfolds':5,
    'seed':42,
    'epsilon':1e-6
}

for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

train_data['Fold'] = -1
kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train_data,y=bins)):
    train_data.loc[valid_idx,'Fold'] = k

In [7]:
train_data.head()

,id,url_legal,license,excerpt,target,standard_error,bins,Fold
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,7,0
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,7,2
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,6,3
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,5,2
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,8,1


## BERT moDEL

In [17]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.targets = df['target'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        target = torch.tensor(self.targets[idx],dtype=torch.float) 
        return encode, target
    
    def __len__(self):
        return len(self.excerpt)

In [18]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))

        score = self.V(att)

        attention_weights = torch.softmax(score, dim=1)

        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [25]:
def run(fold,verbose=True):
    
    def loss_fn(outputs,targets):
        outputs = outputs.view(-1)
        targets = targets.view(-1)
        return torch.sqrt(nn.MSELoss()(outputs,targets))
    
    def train_and_evaluate_loop(train_loader,valid_loader,model, loss_fn, device,optimizer,epoch,fold,best_loss,valid_step=10,lr_scheduler=None):
        train_loss = 0
        for i, (inputs1,targets1) in enumerate(train_loader):
            model.train()
            optimizer.zero_grad()
            inputs1 = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs1.items()}
            targets1 = targets1.to(device)
            outputs1 = model(**inputs1)
            loss1 = loss_fn(outputs1,targets1)
            loss1.backward()
            optimizer.step()
            
            train_loss += loss1.item()
            
            if lr_scheduler:
                lr_scheduler.step()
            
            #evaluating for every valid_step
            if (i % valid_step == 0) or (i == (len(train_loader)-1)):
                model.eval()
                valid_loss = 0
                with torch.no_grad():
                    for j, (inputs2,targets2) in enumerate(valid_loader):
                        inputs2 = {key:val.reshape(val.shape[0],-1).to(device) for key,val in inputs2.items()}
                        targets2 = targets2.to(device)
                        outputs2 = model(**inputs2)
                        loss2 = loss_fn(outputs2,targets2)
                        valid_loss += loss2.item()
                     
                    valid_loss /= len(valid_loader)
                    if valid_loss <= best_loss:
                        if verbose:
                            print(f"epoch:{epoch} | Train Loss:{train_loss/(i+1)} | Validation loss:{valid_loss}")
                            print(f"{g_}Validation loss Decreased from {best_loss} to {valid_loss}{sr_}")

                        best_loss = valid_loss

                        if not os.path.exists(OUTPUT+f'model{fold}'):
                          os.mkdir(OUTPUT+f'model{fold}')
                        torch.save(model.state_dict(),OUTPUT+f'model{fold}/model{fold}.bin')
                        tokenizer.save_pretrained(OUTPUT+f'model{fold}')
                        
        return best_loss
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                
    x_train,x_valid = train_data.query(f"Fold != {fold}"),train_data.query(f"Fold == {fold}")

    MODEL_PATH = INPUT+'clrp_roberta_base'
    tokenizer = AutoTokenizer.from_pretrained('roberta-base')
    model_config = AutoConfig.from_pretrained(MODEL_PATH)
    model = Model(MODEL_PATH)
    model.to(device)
    train_ds = CLRPDataset(x_train,tokenizer,config['max_len'])
    train_dl = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        shuffle=True,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)

    valid_ds = CLRPDataset(x_valid,tokenizer,config['max_len'])
    valid_dl = DataLoader(valid_ds,
                        batch_size = config["batch_size"],
                        shuffle=False,
                        num_workers = 4,
                        pin_memory=True,
                        drop_last=False)
                        
    use_bertadam = False
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [{
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": config['wd'],
        "lr": config['lr'],
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
        "lr": config['lr'],
    }]

    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=config['lr'],
        eps=config['epsilon'],
        correct_bias=not use_bertadam # bias correction step
    )

#    optimizer = optim.AdamW(model.parameters(),lr=config['lr'],weight_decay=config['wd'])

    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= 10 * len(train_dl))

    print(f"Fold: {fold}")
    best_loss = 9999
    for epoch in range(config["epochs"]):
        print(f"Epoch Started:{epoch}")
        best_loss = train_and_evaluate_loop(train_dl,valid_dl,model,loss_fn,
                                            device,optimizer,epoch,fold,best_loss,
                                            valid_step=config['valid_step'],lr_scheduler=lr_scheduler)

In [26]:
for f in range(config['nfolds']):
    run(f)

UnboundLocalError: ignored